В файлах Data_train.csv и Data_test.csv находятся данные о кошках.

В этом задании предлагается изучить поведение диких и домашних кошек на основе нескольких характеристик. О кошках имеется некоторая базовая информация (type, group). У кошек были инструкторы. Инструктор обеспечивает кошку питанием, а некоторых кошек обучает проходить полосу препятствий (некоторых - не обучает). Результаты прохождения полосы препятствий оценивались независимо тремя судьями по стобалльной шкале.

Описание столбцов:

* type - тип кошки: дикая (wild) или домашняя (domestic)

* group - закодированная возрастная группа кошки 

* education - уровень подготовки инструктора

* meal - тип рациона кошки

* preparation course - обучалась ли кошка прохождению полосы препятствий (проходила ли специальный курс)

* score-1 - балл первого судьи за прохождение кошкой полосы препятствий

* score-2 - балл второго судьи за прохождение кошкой полосы препятствий

* score-3 - балл третьего судьи за прохождение кошкой полосы препятствий

Далее за задания можно получить максимум 4 балла.


Считайте данные в два pandas dataframe: df_train и df_test.

In [95]:
import csv
import os
import numpy as np
import pandas as pd

df_train = pd.read_csv(os.path.abspath("Data_train.csv"), sep = ',')
df_train.head()

,type,group,education,meal,preparation course,score-1,score-2,score-3
0,wild,group B,some high school,standard,completed,63,67,67
1,domestic,group C,bachelor's degree,standard,none,67,69,75
2,domestic,group C,some college,standard,completed,69,90,88
3,domestic,group B,some high school,standard,none,62,64,66
4,wild,group E,some college,reduced,none,93,90,83


In [96]:
df_test = pd.read_csv(os.path.abspath("Data_test.csv"), sep = ',')
df_test.head()

,type,group,education,meal,preparation course
0,wild,group D,some college,standard,none
1,wild,group C,master's degree,reduced,completed
2,wild,group B,some college,reduced,completed
3,wild,group A,associate's degree,reduced,none
4,domestic,group B,some college,standard,completed


### Задание 1 (0.25 балла). 
Заполните пропуски в столбце уникальной категорией, если столбец с пропуском категориальный, и средним значением, если столбец числовой. Заполняйте одновременно и df_train, и df_test - одинаковым образом. В ответе укажите количество различных значений, потребовавшихся для заполнения пропусков (это равно количеству новых уникальных категорий плюс количество средних значений для заполнения пропусков в числовых столбцах).

In [97]:
# Заполнение пропусков
# Для категориальных столбцов - новая уникальная категория
# Для числовых столбцов - среднее значение
# .loc[:, df_test.isna().any()]
cols_train = df_train.dtypes.to_string()
cols_test = df_test.dtypes.to_string()
print(cols_train)
print(cols_test)

counter = 0
cols_for_filling = []

# Категориальные столбцы
cat_cols_train = df_train.select_dtypes(include=['object']).columns
for col in cat_cols_train:
    if df_train[col].isna().sum() > 0:
        df_train[col] = df_train[col].fillna('missing')
        counter += 1
        cols_for_filling.append(col)
    
cat_cols_test = df_test.select_dtypes(include=['object']).columns
for col in cat_cols_test:
    if df_test[col].isna().sum() > 0:
        print(col)
        print(df_test[col].isna().sum())
        df_test[col] = df_test[col].fillna('missing')
        counter += 1
        cols_for_filling.append(col)

# Числовые столбцы    
num_cols_train = df_train.select_dtypes(include=['float64', 'int64']).columns
for col in num_cols_train:
    if df_train[col].isna().sum() > 0:
        df_train[col] = df_train[col].fillna(df_train[col].mean())
        counter += 1
        cols_for_filling.append(col)
    
num_cols_test = df_test.select_dtypes(include=['float64', 'int64']).columns
for col in num_cols_test:   
    if df_test[col].isna().sum() > 0:
        df_test[col] = df_test[col].fillna(df_test[col].mean())
        counter += 1
        cols_for_filling.append(col)

print(cols_for_filling)
# Количество различных значений, потребовавшихся для заполнения пропусков
# num_unique_values = len(cat_cols_train) + len(num_cols_train) + len(cat_cols_test) + len(num_cols_test)
print(f"Количество различных значений для заполнения пропусков: {counter}")

type                  object
group                 object
education             object
meal                  object
preparation course    object
score-1                int64
score-2                int64
score-3                int64
type                  object
group                 object
education             object
meal                  object
preparation course    object
group
12
['group', 'group']
Количество различных значений для заполнения пропусков: 2


### Задание 2 (0.3 баллов). 
Кошка прошла полосу препятствий по мнению судьи, если он поставил ей больше 50 баллов. Кошка считается прошедшей полосу препятствий, если все судьи поставили ей больше 50 баллов. В df_train создайте колонку 'Pass' и запишите в неё 1, если кошка прошла полосу препятствий, и 0 иначе. В ответ запишите, сколько кошек из df_train не прошли полосу препятствий.

В df_test от вас скрыта информация о судейских баллах, поэтому неизвестно, прошла кошка полосу препятствия или нет - это и надо будет предсказать в заданиях ниже.

In [98]:
# Создаем колонку 'Pass' в df_train
df_train['Pass'] = ((df_train['score-1'] > 50) & (df_train['score-2'] > 50) & (df_train['score-3'] > 50)).astype(int)

# Количество кошек, не прошедших полосу препятствий
num_not_passed = (df_train['Pass'] == 0).sum()
print(f"Количество кошек, не прошедших полосу препятствий: {num_not_passed}")


Количество кошек, не прошедших полосу препятствий: 145


Это задание выполняйте по данным df_train.

1) Среди всех диких кошек найдите долю кошек, прошедших полосу препятствий. Такую же долю рассчитайте для домашних кошек. В ответе укажите модуль разности этих долей. Ответ округлите до сотых.

In [99]:
# Доля диких кошек, прошедших полосу препятствий
wild_passed = df_train.loc[df_train['type'] == 'wild', 'Pass'].mean()

# Доля домашних кошек, прошедших полосу препятствий 
domestic_passed = df_train.loc[df_train['type'] == 'domestic', 'Pass'].mean()

# Модуль разности долей
diff_abs = abs(wild_passed - domestic_passed)
print(f"Модуль разности долей: {diff_abs:.2f}")

Модуль разности долей: 0.02


2) Сколько кошек среди не прошедших полосу препятствий имели инструктора с уровнем образования "high school"?

In [100]:
not_passed_high_school = df_train.loc[(df_train['Pass'] == 0) & (df_train['education'] == 'high school'), 'education'].count()
print(f"Количество кошек, не прошедших полосу препятствий, с инструктором уровня 'high school': {not_passed_high_school}")

Количество кошек, не прошедших полосу препятствий, с инструктором уровня 'high school': 35


3) Сколько диких кошек среди прошедших полосу препятствий не проходили специальный курс подготовки?

In [101]:
wild_passed_no_prep = df_train.loc[(df_train['type'] == 'wild') & (df_train['Pass'] == 1) & (df_train['preparation course'] != 'Completed'), 'type'].count()
print(f"Количество диких кошек, прошедших полосу препятствий, без специальной подготовки: {wild_passed_no_prep}")

Количество диких кошек, прошедших полосу препятствий, без специальной подготовки: 268


4) Чему равна медиана баллов, выставленных первым судьей?

In [102]:
median_score1 = df_train['score-1'].median()
print(f"Медиана баллов первого судьи: {median_score1}")

Медиана баллов первого судьи: 66.0


5) Найдите межквартильный размах баллов третьего судьи (третья квартиль минус первая квартиль) для домашних кошек, не проходивших специальный курс подготовки.

Комментарий: для вычисления квартилей дискретного распределения используйте интерполяцию меньшим значением (lower interpolation). Это означает, что если искомая квартиль лежит между двумя измерениями i и j, то значение квартили равно i. 

In [103]:
domestic_no_prep = df_train.loc[(df_train['type'] == 'domestic') & (df_train['preparation course'] != 'Completed'), 'score-3']
q1 = domestic_no_prep.quantile(0.25)
q3 = domestic_no_prep.quantile(0.75)
iqr = q3 - q1
print(f"Межквартильный размах баллов третьего судьи: {iqr}")

Межквартильный размах баллов третьего судьи: 20.0


Задание 4 (0.7 баллов). 

a) (0.3 баллa). Далее используйте только категориальные столбцы. Закодируйте их с помощью One-hot encoding с учетом того, что мы не хотим получить мультиколлинеарности в новых данных. Сколько получилось числовых столбцов из исходных категориальных? Кодируйте и df_train, и df_test.


In [104]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encoding для df_train
categorical_cols_train = df_train.select_dtypes(include=['object']).columns
categorical_cols_test = df_test.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(df_train[categorical_cols_train])
df_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(categorical_cols_train))

# One-hot encoding для df_test
X_test_encoded = encoder.transform(df_test[categorical_cols_test])
df_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(categorical_cols_test))

# Количество числовых столбцов
num_numeric_cols = len(encoder.get_feature_names_out(categorical_cols_train)) + len(encoder.get_feature_names_out(categorical_cols_test))
print(f"Количество числовых столбцов: {num_numeric_cols}")


Количество числовых столбцов: 36


D:\Programming\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


б) (0.4 балла). Попытаемся по характеристикам кошки (бывшие категориальные, а теперь - числовые столбцы) предсказать, прошла она полосу препятствий или нет.


Сформируйте из df_train матрицу объект-признак X и вектор ответов y. 


Обучите решающее дерево (DecisionTreeClassifier из библиотеки sklearn.tree) глубины 5 с энтропийным критерием информативности на закодированных в пункте а) тренировочных данных по кросс-валидации с тремя фолдами, метрика качества - roc-auc.


Чему равен roc-auc, усредненный по фолдам? Ответ округлите до десятых.


Комментарий: остальные гиперпараметры дерева оставьте дефолтными (splitter='best', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

In [105]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# Создаем матрицу объект-признак X и вектор ответов y
X = df_train_encoded.values
y = df_train['Pass'].values

# Обучение решающего дерева с кросс-валидацией
model = DecisionTreeClassifier(max_depth=5, criterion='entropy')
roc_auc_scores = cross_val_score(model, X, y, cv=3, scoring='roc_auc')

# Усредненный roc-auc
avg_roc_auc = np.mean(roc_auc_scores)
print(f"Усредненный roc-auc: {avg_roc_auc:.1f}")

Усредненный roc-auc: 0.7


Задание 6 (1.5 балла максимум). 


a) (0.25 балла). Подберите глубину решающего дерева (max_depth), перебирая глубину от 2 до 20 с шагом 1 и используя перебор по сетке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - roc-auc. В ответ запишите наилучшее среди искомых значение max_depth.


Комментарий: остальные гиперпараметры дерева оставьте дефолтными (splitter='best', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

In [106]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Матрица объект-признак и вектор ответов
X = df_train_encoded.values
y = df_train['Pass'].values

# Перебор глубины дерева
param_grid = {'max_depth': np.arange(2, 21, 1)}
model = DecisionTreeClassifier(criterion='entropy')
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='roc_auc')
grid_search.fit(X, y)

# Наилучшее значение max_depth
best_max_depth = grid_search.best_params_['max_depth']
print(f"Наилучшее значение max_depth: {best_max_depth}")


Наилучшее значение max_depth: 2


б) (0.5 балла). Добавьте к данным новый признак cat_bio, содержащий в качестве значений пары значений из столбца type и столбца group. Например, если кошка имеет type='wild' и  group='group B', то в cat_bio будет стоять строка '(wild, group B)'. Примените OneHotEncoding (с учетом того, что мы не хотим получить мультиколлинеарности в новых данных) к столбцам 'cat_bio', 'education', 'meal', 'preparation course', а затем обучите решающее дерево глубины 5 с энтропийным критерием информативности на полученных после кодирования данных. Чему равен roc-auc? Ответ округлите до сотых.

Комментарий: остальные гиперпараметры дерева оставьте дефолтными (splitter='best', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

In [107]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Создание признака cat_bio
df_train['cat_bio'] = df_train['type'] + ', ' + df_train['group']
df_test['cat_bio'] = df_test['type'] + ', ' + df_test['group']

# One-hot encoding
categorical_cols = ['cat_bio', 'education', 'meal', 'preparation course']
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(df_train[categorical_cols])
df_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(categorical_cols))

X_test_encoded = encoder.transform(df_test[categorical_cols])
df_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(categorical_cols))

# Обучение решающего дерева
X = df_train_encoded.values
y = df_train['Pass'].values
model = DecisionTreeClassifier(max_depth=5, criterion='entropy')
roc_auc_scores = cross_val_score(model, X, y, cv=3, scoring='roc_auc')
avg_roc_auc = np.mean(roc_auc_scores)
print(f"Усредненный roc-auc: {avg_roc_auc:.2f}")


Усредненный roc-auc: 0.68


D:\Programming\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


в) (0.75 балла). Теперь вы можете использовать любую модель машинного обучения для решения задачи. Также можете делать любую другую обработку признаков. Ваша задача - получить наилучшее качество (ROC_AUC).

Качество проверяется на тестовых данных.

ROC_AUC > 0.7 - 0.25 балла
ROC_AUC > 0.74 - 0.75 балла
Сдайте файл result.txt: в файле должна одна колонка с предсказанными значениями целевой переменной для тестовой выборки, без индекса и заголовка.

In [109]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# One-hot encoding для df_train и df_test
categorical_cols = ['type', 'group', 'education', 'meal', 'preparation course']
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(df_train[categorical_cols])
df_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(categorical_cols))

X_test_encoded = encoder.transform(df_test[categorical_cols])
df_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(categorical_cols))

# Логистическая регрессия
lr_model = LogisticRegression()
lr_roc_auc = cross_val_score(lr_model, df_train_encoded.values, df_train['Pass'].values, cv=3, scoring='roc_auc').mean()
print(f"ROC_AUC для логистической регрессии: {lr_roc_auc:.2f}")

# Случайный лес
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_roc_auc = cross_val_score(rf_model, df_train_encoded.values, df_train['Pass'].values, cv=3, scoring='roc_auc').mean()
print(f"ROC_AUC для случайного леса: {rf_roc_auc:.2f}")

# Выбор лучшей модели
best_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
best_model.fit(df_train_encoded.values, df_train['Pass'].values)
y_pred = best_model.predict(df_test_encoded.values)

print("Содержимое файла results.txt:")
print(pd.Series(y_pred).to_string(index=False, header=False))

D:\Programming\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ROC_AUC для логистической регрессии: 0.73
ROC_AUC для случайного леса: 0.69
Содержимое файла results.txt:
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
0
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1


In [111]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

# Подготовка данных
categorical_cols = ['type', 'group', 'education', 'meal', 'preparation course']
X_train = df_train[categorical_cols]
y_train = df_train['Pass']
X_test = df_test[categorical_cols]

# Обучение модели CatBoost
cat_model = CatBoostClassifier(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    loss_function='Logloss',
    eval_metric='AUC',
    random_state=42
)
cat_model.fit(X_train, y_train)

# Оценка качества на тестовой выборке
y_pred_proba = cat_model.predict_proba(X_test)[:, 1]
roc_auc = cross_val_score(cat_model, X_train, y_train, cv=3, scoring='roc_auc').mean()
print(f"ROC_AUC для CatBoost: {roc_auc:.2f}")

# Сохранение результатов
print("Содержимое файла results.txt:")
print(pd.Series(y_pred_proba).to_string(index=False, header=False))

ModuleNotFoundError: No module named 'catboost'